In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import csv
import pandas as pd
import datetime
import time
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns
from matplotlib import rcParams
from sklearn.cluster import MiniBatchKMeans,KMeans
import math
import pickle
import os
import xgboost as xgb


import warnings
import networkx as nx
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds,eigs
import gc

# 1. Reading Data

In [2]:
if os.path.isfile('data/after_eda/train_pos_after_eda.csv'):
    train_graph=nx.read_edgelist('data/after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
    print(nx.info(train_graph))

Name: 
Type: DiGraph
Number of nodes: 1780722
Number of edges: 7550015
Average in degree:   4.2399
Average out degree:   4.2399


# 2.Similarity measures

# 2.1 Jaccard Distance



In [3]:
#for followees
def jaccard_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a)))==0 | len(set(train_graph.successors(b)))==0:
            return 0
        sim=(len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                  (len(set(train_graph.successors(a)).union(set(train_graph.successors(b)))))
    except:
        return 0;
    return sim

In [4]:
print(jaccard_for_followees(273084,1505602))

0.0


In [5]:
#for followers
def jaccard_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a)))==0 | len(set(train_graph.predecessors(b)))==0:
            return 0
        sim=(len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                  (len(set(train_graph.predecessors(a)).union(set(train_graph.predecessors(b)))))
    except:
        return 0;
    return sim

In [6]:
print(jaccard_for_followers(273084,470294))

0.0


# 2.2 Cosine Distance (Otsuka-Ochiai coefficient)

In [7]:
#for followees
def cosine_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a)))==0 | len(set(train_graph.successors(b)))==0:
            return 0
        sim=(len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                  (math.sqrt(len(set(train_graph.successors(a))*len(set(train_graph.successors(b))))))
    except:
        return 0;
    return sim

In [8]:
print(cosine_for_followees(273084,1505602))

0


In [9]:
#for followers
def cosine_for_followers(a,b):
    try:
        if len(set(train_graph.successors(a)))==0 | len(set(train_graph.successors(b)))==0:
            return 0
        sim=(len(set(train_graph.successors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                  (math.sqrt(len(set(train_graph.predecessors(a)))*len(set(train_graph.predecessors(b))))))
    except:
        return 0;
    return sim

SyntaxError: invalid syntax (<ipython-input-9-61b0c0ccb4f6>, line 7)

In [ ]:
print(cosine_for_followers(2,470294))

# 3.Ranking Measures

https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html

PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links

# 3.1 Page Rank

https://en.wikipedia.org/wiki/PageRank

In [ ]:
if not os.path.isfile('data/fea_sample/page_rank.p'):
    pr=nx.pagerank(train_graph,alpha=0.85)
    filename="data/fea_sample/page_rank.p"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open('data/fea_sample/page_rank.p', 'wb') as f:
        pickle.dump(pr, f)
else:
    pr=pickle.load(open('data/fea_sample/page_rank.p','rb'))

In [ ]:
print('min',pr[min(pr,key=pr.get)])
print('max',pr[max(pr,key=pr.get)])
print('mean',float(sum(pr.values()))/len(pr))

In [ ]:
#for inputing to nodes which are not there in Train data
mean_pr=float(sum(pr.values())/len(pr))
print(mean_pr)

# Graph based features

# 4.1 Shortest Path

Getting Shortest path between two nodes,if nodes have an edge i.e trivially connected then we are removing that edge and calculating the shortest path

In [ ]:
#if has direct edge then deleting that edge and calculating shortest path
def compute_shortest_path_length(a,b):
    p=-1
    try:
        if(train_graph.has_edge(a,b)):
            train_graph.remove_edge(a,b)
            p=nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p=nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
         return -1

In [ ]:
#test
compute_shortest_path_length(77697,826021)

# 4.2 Checking for same weakly connected component(Community)

In [ ]:
#getting weekly connected edges from graph
wcc=list(nx.weakly_connected_components(train_graph))
def belongs_to_same_wcc(a,b):
    index=[]
    if train_graph.has_edge(b,a):
        return 1
    if train_graph.has_edge(a,b):
        for i in wcc:
            if a in i:
                index=i
                break
        if( b in index):
            train_graph.remove_edge(a,b)
            if compute_shortest_path_length(a,b)==-1:
                train_graph.add_edge(a,b)
                return 0
            else:
                train_graph.add_edge(a,b)
                return 1
    else:
        for i in wcc:
            if a in i:
                index=i
                break
            if (b in index):
                return 1
            else:
                 return 0
        

In [ ]:
belongs_to_same_wcc(861,1659750)

In [ ]:
belongs_to_same_wcc(669354,1635354)

# Adamic/Adar Index

Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices

https://en.wikipedia.org/wiki/Adamic/Adar_index

In [ ]:
#adar index
def calc_adar_in(a,b):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(train_graph.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

In [ ]:
calc_adar_in(1,189226)

# follow back feature


In [ ]:
def follows_back(a,b):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0


In [ ]:
follows_back(1,189226)

In [ ]:
follows_back(6693,1635354)

# Katz Centrality

https://www.geeksforgeeks.org/katz-centrality-centrality-measure/

In [ ]:
if not os.path.isfile('data/fea_sample/katz.p'):
    katz=nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
    pickle.dump(katz,open('data/fea_sample/katz.p','wb'))
else:
    katz=pickle.load(open('data/fea_sample/katz.p','rb'))

In [ ]:
print('min',katz[min(katz,key=katz.get)])
print('max',katz[max(katz,key=katz.get)])
print('mean',float(sum(katz.values()))/len(katz))